In [2]:
#Revisar esto para representar los grafos generados.

def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

In [1]:
#Importamos lo necesario para la GNN

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.explain import Explainer
from torch_geometric.explain.algorithm import GNNExplainer
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
#from torchvision import transforms, models
#from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch import optim
import numpy as np


%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx


Dependiendo de dónde querramos trabajar corremos un bloque u otro

In [2]:
import multiprocessing

num_workers = max(4, multiprocessing.cpu_count())  # Usa hasta 4 workers o el máximo de la CPU, sin embargo, es mejor que el número máximo coincida con los núcleos de la CPU, no con los procesadores lógicos
print(f"Usando {num_workers} workers para cargar los datos")
enable_cpu_affinity = True #Revisar bien como funciona esto


Usando 12 workers para cargar los datos


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Importamos los módulos a usar, incluyendo GNNExplainer para poder observar más adelante si nuestra GNN está bien planteada, permitiendo entender el motivo de su predicción.

In [24]:
from Dataset_loader import PCBDataset #

# Ruta al archivo .pth (relativa a tu ubicación actual)
file_path = 'PCB_dataset.pth'
dataset = load_dataset(file_path)

print(f"Dataset cargado exitosamente. Tamaño del dataset: {len(dataset)}")
print(type(dataset))
print(file_path)

Dataset cargado exitosamente. Tamaño del dataset: 20000
<class 'Dataset_loader.PCBDataset'>
PCB_dataset.pth


In [14]:
import os
print(os.listdir())



['.gitkeep', 'Dataset_loader.py', 'GNN_Main.ipynb', 'PCB_dataset.pth', 'pruebaGNN.ipynb', 'pruebascpu.ipynb', 'temp-plot.html', '__init__.py', '__pycache__']


c:\Users\ramse\Escritorio\Aero Ordenador\4to Aero\2do Cuatri\Repo_TFG\ia_thermal\ernestoemedina
False
['.gitkeep', 'Dataset_loader.py', 'GNN_Main.ipynb', 'PCB_dataset.pth', 'pruebaGNN.ipynb', 'pruebascpu.ipynb', 'temp-plot.html', '__init__.py', '__pycache__']


In [ ]:
# Define edge index (source, target) pairs
edge_index = torch.tensor([
    [0, 1, 1, 2, 2, 3],  # Source nodes
    [1, 0, 2, 1, 3, 2]   # Target nodes
], dtype=torch.long)  ## Podrias probar a hacer un bucle que vaya sobreescribiendo

# Define node features (4 nodes, each with 3 features)
x = torch.tensor([
    [1, 0, 1], 
    [0, 1, 1], 
    [1, 1, 0], 
    [0, 0, 1]
], dtype=torch.float)

# Define node labels (for classification)
y = torch.tensor([0, 1, 0, 1], dtype=torch.long)


# Comprobamos ciertas cosas de PCB_dataset
#print(len(dataset))


# Create PyG data object
data = Data(x=x, edge_index=edge_index, y=y).to(device)
data.validate(raise_on_error=True) # Para validar que los datos son correctos
print("Numero de nodos", data.num_nodes)
print("Numero de edges", data.num_edges)
print("Numero de features", data.num_node_features)
data.has_isolated_nodes()
data.has_self_loops()
data.is_directed()

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Instantiate model
model = GCN(in_channels=data.num_node_features, hidden_channels=4,  out_channels=data.num_node_features).to(device) 
# Las salidas y entradas son las mismas, ya que son las temperaturas y potencias "predichas" por el modelo


optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss().to(device)


# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Train for 200 epochs
for epoch in range(200):
    loss = train()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Revisar lo del explainer porque creo que esta mal, que ahora se llama de otra manera (fijarse abajo)
explainer = GNNExplainer(model, epochs=200)

# Explain the prediction for node 2
node_idx = 2
explanation = explainer.explain_node(node_idx, data.x, data.edge_index)

# Visualize the explanation
ax, G = explainer.visualize_subgraph(node_idx, data.edge_index, explanation.edge_mask, threshold=0.5)
plt.show()


model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)  # Get predicted class
print("Predicted classes:", pred.tolist())


Numero de nodos 4
Numero de edges 6
Numero de features 3
Epoch 0, Loss: 0.7374
Epoch 20, Loss: 0.6704
Epoch 40, Loss: 0.6580
Epoch 60, Loss: 0.6424
Epoch 80, Loss: 0.6232
Epoch 100, Loss: 0.6000
Epoch 120, Loss: 0.5760
Epoch 140, Loss: 0.5503
Epoch 160, Loss: 0.5259
Epoch 180, Loss: 0.5021


TypeError: GNNExplainer.__init__() got multiple values for argument 'epochs'

In [28]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.explain import Explainer, ModelConfig
from torch_geometric.explain.config import ModelMode, ModelTaskLevel, ModelReturnType
from torch_geometric.explain.algorithm import GNNExplainer
import matplotlib.pyplot as plt

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model configuration
model_config = ModelConfig(
    mode=ModelMode.classification,     # Use ModelMode.classification
    task_level=ModelTaskLevel.node,    # Use ModelTaskLevel.node
    return_type=ModelReturnType.log_probs # Use ModelReturnType.log_probs
)
# Define edge index (source, target) pairs
edge_index = torch.tensor([
    [0, 1, 1, 2, 2, 3],  # Source nodes
    [1, 0, 2, 1, 3, 2]   # Target nodes
], dtype=torch.long).to(device)

# Define node features (4 nodes, each with 3 features)
x = torch.tensor([
    [1, 0, 1], 
    [0, 1, 1], 
    [1, 1, 0], 
    [0, 0, 1]
], dtype=torch.float).to(device)

# Define node labels (for classification)
y = torch.tensor([0, 1, 0, 1], dtype=torch.long).to(device)

# Create PyG data object
data = Data(x=x, edge_index=edge_index, y=y).to(device)
data.validate(raise_on_error=True)

# Check data properties
print("Number of nodes:", data.num_nodes)
print("Number of edges:", data.num_edges)
print("Number of features per node:", data.num_node_features)
print("Number of classes:", len(torch.unique(data.y)))

# Define GCN Model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Instantiate model
model = GCN(in_channels=data.num_node_features, hidden_channels=4, 
            out_channels=len(torch.unique(data.y))).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss().to(device)

# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Train for 200 epochs
for epoch in range(200):
    loss = train()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Model evaluation
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
print("Predicted classes:", pred.tolist())

# Initialize the Explainer with the GNNExplainer algorithm
explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(num_epochs=200),
    explanation_type='model',
    model_config=model_config,
    node_mask_type='object',
    edge_mask_type='object'
)

# Explain the prediction for a specific node (e.g., node 2)
node_idx = 2
explanation = explainer(data.x, data.edge_index, index=node_idx)

# Visualize the explanation
fig, ax = explanation.visualize_graph()
plt.show()



AttributeError: type object 'ModelMode' has no attribute 'classification'